In [3]:
!pip install 'aif360[All]'

  Using cached aif360-0.6.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached skorch-1.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached sphinx_rtd_theme-3.0.2-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached igraph-0.11.8-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached lime-0.2.0.1.tar.gz (275 kB)
  Preparing metadata (setup.py) ... done
  Using cached fairlearn-0.11.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached inFairness-0.2.3-py3-none-any.whl.metadata (8.1 kB)
  Using cached pytest_cov-6.0.0-py3-none-any.whl.metadata (27 kB)
  Using cached ipympl-0.9.4-py3-none-any.whl.metadata (8.7 kB)
  Using cached adversarial_robustness_toolbox-1.18.2-py3-none-any.whl.metadata (11 kB)
  Using cached BlackBoxAuditing-0.1.54.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached POT-0.9.5-cp

In [4]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    classification_report,
    precision_score,
    confusion_matrix
)
from collections import defaultdict

/usr/local/lib/python3.10/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.10/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [29]:
!wget https://raw.githubusercontent.com/Trusted-AI/AIF360/main/examples/common_utils.py
from common_utils import compute_metrics

--2024-12-02 18:22:11--  https://raw.githubusercontent.com/Trusted-AI/AIF360/main/examples/common_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1340 (1.3K) [text/plain]
Saving to: ‘common_utils.py.1’

common_utils.py.1   100%[===================>]   1.31K  --.-KB/s    in 0s      

2024-12-02 18:22:12 (69.6 MB/s) - ‘common_utils.py.1’ saved [1340/1340]



In [30]:
# Load the Drug Consumption dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data"
columns = [
    "ID", "Age", "Gender", "Education", "Country", "Ethnicity", "Neuroticism", "Extraversion",
    "Openness", "Agreeableness", "Conscientiousness", "Impulsiveness", "Sensation-seeking",
    "Alcohol", "Amphetamines", "Amyl_nitrite", "Benzodiazepines", "Caffeine", "Cannabis",
    "Chocolate", "Cocaine", "Crack", "Ecstasy", "Heroin", "Ketamine", "Legal_highs", "LSD",
    "Methadone", "Mushrooms", "Nicotine", "Semer", "Volatile_substance"
]
data = pd.read_csv(url, header=None, names=columns)

In [31]:
# Define target (Y) and bias group (B)
target = "Cannabis"
bias_group = "Ethnicity"

# Binarize the target and bias group
data[target] = data[target].apply(lambda x: 1 if x in ["CL3", "CL4", "CL5", "CL6"] else 0) #1 if they have used it before/often, 0 if not
data[bias_group] = data[bias_group].apply(lambda x: 1 if x == -0.31685 else 0)   #1 if they are white, 0 if they are not white

In [32]:
# Preprocess categorical features
categorical_columns = data.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_columns:
    encoder = LabelEncoder()
    data[col] = encoder.fit_transform(data[col])
    label_encoders[col] = encoder

# Ensure all columns are numeric
for col in data.columns:
    if data[col].dtype == "object":
        data[col] = pd.to_numeric(data[col], errors="coerce")

In [33]:
# Split dataset into train and test sets
dataset_orig_train, dataset_orig_test = train_test_split(data, test_size=0.3, random_state=42)

# Define features and target for training and testing
X_train = dataset_orig_train.drop(columns=[target, bias_group])
y_train = dataset_orig_train[target]
X_test = dataset_orig_test.drop(columns=[target, bias_group])
y_test = dataset_orig_test[target]

# Convert features to float to avoid type issues
X_train = X_train.astype(float)
X_test = X_test.astype(float)

# Create a BinaryLabelDataset instance with a different variable name
dataset_bld_train = BinaryLabelDataset(df=dataset_orig_train,  # Use the original DataFrame here
                                        label_names=[target],
                                        protected_attribute_names=[bias_group])

privileged_groups = [{bias_group: 1}]
unprivileged_groups = [{bias_group: 0}]

In [34]:
dataset_orig_train = BinaryLabelDataset(df=dataset_orig_train,
                                        label_names=[target],
                                        protected_attribute_names=[bias_group])

privileged_groups = [{bias_group: 1}]
unprivileged_groups = [{bias_group: 0}]

metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.071082


In [35]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(dataset_orig_train)
dataset_transf_train = RW.transform(dataset_orig_train)
assert np.abs(dataset_transf_train.instance_weights.sum()-dataset_orig_train.instance_weights.sum())<1e-6

In [36]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train,
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.000000


In [37]:
print("Original training dataset shape:", dataset_orig_train.features.shape)
print("Original testing dataset shape:", dataset_orig_test.shape)

Original training dataset shape: (1319, 31)
Original testing dataset shape: (566, 32)


In [38]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, group, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    print(f"\n{model_name} - Regular Data")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Balanced Accuracy: {balanced_acc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # Fairness metrics
    equalized_odds = calculate_equalized_odds(y_test, y_pred, group)
    predictive_parity = calculate_predictive_value_parity(y_test, y_pred, group)

    print("\nEqualized Odds (TPR, FPR) by Group:")
    for g, metrics in equalized_odds.items():
        print(f"Group {g}: TPR = {metrics['TPR']:.2f}, FPR = {metrics['FPR']:.2f}")

    print("\nPredictive Parity (PPV) by Group:")
    for g, ppv in predictive_parity.items():
        print(f"Group {g}: PPV = {ppv:.2f}")

    return y_pred

In [39]:
def calculate_equalized_odds(y_true, y_pred, group):
    tpr_fpr = defaultdict(dict)
    unique_groups = np.unique(group)

    for g in unique_groups:
        mask = group == g
        tn, fp, fn, tp = confusion_matrix(y_true[mask], y_pred[mask]).ravel()
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # Fall-out
        tpr_fpr[g] = {"TPR": tpr, "FPR": fpr}

    return tpr_fpr

In [40]:
def calculate_predictive_value_parity(y_true, y_pred, group):
    ppv = {}
    unique_groups = np.unique(group)

    for g in unique_groups:
        mask = group == g
        precision = precision_score(y_true[mask], y_pred[mask], zero_division=0)
        ppv[g] = precision

    return ppv

In [41]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the fitted scaler
X_test_scaled = scaler.transform(X_test)

# Then call the train_and_evaluate_model function with the scaled data
knn_regular = KNeighborsClassifier(n_neighbors=5)
y_pred_knn_regular = train_and_evaluate_model(
    knn_regular, X_train_scaled, y_train, X_test_scaled, y_test, dataset_orig_test[bias_group], "KNN Model"
)



KNN Model - Regular Data
Accuracy: 0.83
Balanced Accuracy: 0.83

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.86      0.82       249
           1       0.88      0.80      0.84       317

    accuracy                           0.83       566
   macro avg       0.83      0.83      0.83       566
weighted avg       0.83      0.83      0.83       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.90, FPR = 0.05
Group 1: TPR = 0.79, FPR = 0.15

Predictive Parity (PPV) by Group:
Group 0: PPV = 0.96
Group 1: PPV = 0.87


In [42]:
rf_regular = RandomForestClassifier(random_state=42)
y_pred_rf_regular = train_and_evaluate_model(
    rf_regular, X_train, y_train, X_test, y_test, dataset_orig_test[bias_group], "Random Forest Model"
)


Random Forest Model - Regular Data
Accuracy: 0.88
Balanced Accuracy: 0.88

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.86       249
           1       0.89      0.90      0.89       317

    accuracy                           0.88       566
   macro avg       0.88      0.88      0.88       566
weighted avg       0.88      0.88      0.88       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.93, FPR = 0.05
Group 1: TPR = 0.89, FPR = 0.15

Predictive Parity (PPV) by Group:
Group 0: PPV = 0.97
Group 1: PPV = 0.88


In [43]:
logistic_regular = LogisticRegression(random_state=42, max_iter=1000)
y_pred_logistic_regular = train_and_evaluate_model(
    logistic_regular, X_train_scaled, y_train, X_test_scaled, y_test, dataset_orig_test[bias_group], "Logistic Regression Model"
)


Logistic Regression Model - Regular Data
Accuracy: 0.85
Balanced Accuracy: 0.85

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.86      0.83       249
           1       0.89      0.83      0.86       317

    accuracy                           0.85       566
   macro avg       0.84      0.85      0.85       566
weighted avg       0.85      0.85      0.85       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.93, FPR = 0.05
Group 1: TPR = 0.82, FPR = 0.14

Predictive Parity (PPV) by Group:
Group 0: PPV = 0.97
Group 1: PPV = 0.88


In [44]:
feature_columns = dataset_orig_test.columns[1:]  # Exclude the 'ID' column, assuming it's the first

# Select the same columns from dataset_orig_test to create X_test_rw
X_test_rw = dataset_orig_test[feature_columns].astype(float)

# Scale the data
scaler = StandardScaler()
X_train_scaled_rw = scaler.fit_transform(X_train_rw)
X_test_scaled_rw = scaler.transform(X_test_rw)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [45]:
knn_reweighted = KNeighborsClassifier(n_neighbors=5)
y_pred_knn_reweighted = train_and_evaluate_model(
    knn_reweighted, X_train_scaled_rw, y_train, X_test_scaled_rw, y_test, dataset_orig_test[bias_group], "KNN Model - reweighted"
)


KNN Model - reweighted - Regular Data
Accuracy: 0.75
Balanced Accuracy: 0.77

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.89      0.76       249
           1       0.88      0.65      0.75       317

    accuracy                           0.75       566
   macro avg       0.77      0.77      0.75       566
weighted avg       0.79      0.75      0.75       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.77, FPR = 0.00
Group 1: TPR = 0.63, FPR = 0.12

Predictive Parity (PPV) by Group:
Group 0: PPV = 1.00
Group 1: PPV = 0.87


In [46]:

logistic_reweighted = LogisticRegression(random_state=42, max_iter=1000)
y_pred_logistic_reweighted = train_and_evaluate_model(
    logistic_reweighted, X_train_scaled_rw, dataset_transf_train.labels.ravel(), X_test_scaled_rw, y_test, dataset_orig_test[bias_group], "Logistic Regression Model - Reweighted"
)


Logistic Regression Model - Reweighted - Regular Data
Accuracy: 0.78
Balanced Accuracy: 0.77

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.71      0.74       249
           1       0.79      0.83      0.81       317

    accuracy                           0.78       566
   macro avg       0.78      0.77      0.78       566
weighted avg       0.78      0.78      0.78       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.93, FPR = 0.35
Group 1: TPR = 0.82, FPR = 0.28

Predictive Parity (PPV) by Group:
Group 0: PPV = 0.80
Group 1: PPV = 0.79


In [47]:
# Assuming dataset_transf_train and dataset_orig_test are already defined and processed
# Extract features and labels from dataset_transf_train
X_train_rw = dataset_transf_train.features
y_train = dataset_transf_train.labels.ravel()  # Flatten labels

# Extract features from dataset_orig_test
feature_columns = dataset_orig_test.columns[1:]  # Exclude the 'ID' column, assuming it's the first
X_test_rw = dataset_orig_test[feature_columns].astype(float)

# Scale the data
scaler = StandardScaler()
X_train_scaled_rw = scaler.fit_transform(X_train_rw)
X_test_scaled_rw = scaler.transform(X_test_rw)

rf_reweighted = RandomForestClassifier(random_state=42)
y_pred_rf_reweighted = train_and_evaluate_model(
    rf_reweighted, X_train_scaled_rw, y_train, X_test_scaled_rw, y_test, dataset_orig_test[bias_group], "Random Forest Model - Reweighted"
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



Random Forest Model - Reweighted - Regular Data
Accuracy: 0.81
Balanced Accuracy: 0.82

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.90      0.81       249
           1       0.90      0.74      0.82       317

    accuracy                           0.81       566
   macro avg       0.82      0.82      0.81       566
weighted avg       0.83      0.81      0.81       566


Equalized Odds (TPR, FPR) by Group:
Group 0: TPR = 0.87, FPR = 0.05
Group 1: TPR = 0.73, FPR = 0.11

Predictive Parity (PPV) by Group:
Group 0: PPV = 0.96
Group 1: PPV = 0.89
